In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd
from datasets import load_dataset

2024-09-14 16:28:57.444463: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 16:28:57.447398: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 16:28:57.455915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 16:28:57.469404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 16:28:57.473102: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 16:28:57.483636: I tensorflow/core/platform/cpu_feature_gu

In [2]:
ds = load_dataset("mrm8488/CHISTES_spanish_jokes")

In [3]:
df = ds['train'].to_pandas()

In [4]:
text = ''
for i in df['text']:
    i = i.lower()
    i = i.replace('-', '')
    i = i.replace('–', '')
    i = i.replace('á', 'a')
    i = i.replace('é', 'e')
    i = i.replace('í', 'i')
    i = i.replace('ó', 'o')
    i = i.replace('ú', 'u')
    i = i.replace('ü', 'u')
    i = i.replace('ñ', 'gn')
    i = i.replace('¿', '')
    i = i.replace('¡', '')
    i = i.replace('\n ', '\n')
    text += '$' + i 
text = text[2:]

In [5]:
chars = tf.strings.unicode_split(text, input_encoding='UTF-8')

In [6]:
vocab = sorted(set(text))

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.Tensor: shape=(660873,), dtype=int64, numpy=array([48, 31, 46, ..., 42, 31,  3])>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [10]:
chars = chars_from_ids(ids)
chars

<tf.Tensor: shape=(660873,), dtype=string, numpy=array([b'r', b'a', b'p', ..., b'l', b'a', b'!'], dtype=object)>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

b'rapido, necesitamos sangre!\nyo soy 0 positivo.\npues muy mal, necesitamos una mentalidad optimista.$ cual es el mejor portero del mundial? \nevidente el de paraguay!$el otro dia unas chicas llamaron a mi puerta y me pidieron una pequegna donacion para una piscina local. \nles di un garrafa de agua.$ andresito, que planeta va despues de marte? \nmiercole, segnorita.$ por que bob esponja no va al gimnasio? \nporque ya esta cuadrado.$van dos ciegos y le dice uno al otro: \nojala lloviera... \nojala yo tambien...$noticia de ultima hora!! \n\nmuere una suegra atropellada en canarias. \n\ny esto es todo, las 8 en espagna y una menos en canarias...$ mama, mama, en el colegio dicen que estoy loco.\ny quien dice eso de ti?\n...me lo dicen las sillas...$ mama, mama, me haces un bocata de jamon?\nyork?\nsi, turk.$ que pasa si te expulsan de cuatro univerdades?\n....\nque estas perdiendo facultades$ que es un pelo en una cama?\n...\nel bello durmiente$ que le dice el techo del comedor al techo 

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(660873,), dtype=int64, numpy=array([48, 31, 46, ..., 42, 31,  3])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

r
a
p
i
d
o
,
 
n
e


2024-09-14 16:29:07.734062: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'r' b'a' b'p' b'i' b'd' b'o' b',' b' ' b'n' b'e' b'c' b'e' b's' b'i'
 b't' b'a' b'm' b'o' b's' b' ' b's' b'a' b'n' b'g' b'r' b'e' b'!' b'\n'
 b'y' b'o' b' ' b's' b'o' b'y' b' ' b'0' b' ' b'p' b'o' b's' b'i' b't'
 b'i' b'v' b'o' b'.' b'\n' b'p' b'u' b'e' b's' b' ' b'm' b'u' b'y' b' '
 b'm' b'a' b'l' b',' b' ' b'n' b'e' b'c' b'e' b's' b'i' b't' b'a' b'm'
 b'o' b's' b' ' b'u' b'n' b'a' b' ' b'm' b'e' b'n' b't' b'a' b'l' b'i'
 b'd' b'a' b'd' b' ' b'o' b'p' b't' b'i' b'm' b'i' b's' b't' b'a' b'.'
 b'$' b' ' b'c'], shape=(101,), dtype=string)


2024-09-14 16:29:07.780794: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'rapido, necesitamos sangre!\nyo soy 0 positivo.\npues muy mal, necesitamos una mentalidad optimista.$ c'
b'ual es el mejor portero del mundial? \nevidente el de paraguay!$el otro dia unas chicas llamaron a mi '
b'puerta y me pidieron una pequegna donacion para una piscina local. \nles di un garrafa de agua.$ andre'
b'sito, que planeta va despues de marte? \nmiercole, segnorita.$ por que bob esponja no va al gimnasio? '
b'\nporque ya esta cuadrado.$van dos ciegos y le dice uno al otro: \nojala lloviera... \nojala yo tambien.'


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'rapido, necesitamos sangre!\nyo soy 0 positivo.\npues muy mal, necesitamos una mentalidad optimista.$ '
Target: b'apido, necesitamos sangre!\nyo soy 0 positivo.\npues muy mal, necesitamos una mentalidad optimista.$ c'


2024-09-14 16:29:07.927668: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 100

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [51]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x)
        
        # Corregir el estado inicial si es None
        if states is None:
            batch_size = tf.shape(inputs)[0]  # Obtén el tamaño de lote desde los inputs
            states = tf.zeros([batch_size, self.gru.units])  # Estado inicial con ceros y forma [batch_size, rnn_units]
        
        # Llama a la GRU con el estado inicial
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x


In [52]:
# Instanciación del modelo
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

# Ejecución del modelo en un lote de datos
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape,
          "# (batch_size, sequence_length, vocab_size)")

(64, 100, 94) # (batch_size, sequence_length, vocab_size)
